# Delta Water Unavailability Methodology Module 1: Supply Processing

In [ ]:
# This script takes user inputs and calls published supply datasets in the Sacramento-San Joaquin Delta Watershed
# to prepare them for the water unavailability analysis.
# Further documentation is available on the Delta Water Unavailability Methodology website:
# https://www.waterboards.ca.gov/waterrights/water_issues/programs/drought/drought_tools_methods/delta_method.html

# Import necessary packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from dateutil.parser import parse
from dateutil.relativedelta import relativedelta
import calendar
import os

In [ ]:
# Function to create output folders if they don't already exist
def create_folder(folder_path):
    # Check if the folder exists
    if not os.path.exists(folder_path):
        
        # If not, create the folder
        os.makedirs(folder_path)
        print(f"Folder '{folder_path}' created successfully.")
    else:
        print(f"Folder '{folder_path}' already exists.")

# Create intermediate-outputs folder
folder_path = './intermediate-outputs'
create_folder(folder_path)

# Create output-data folder
folder_path = './output-data'
create_folder(folder_path)

In [ ]:
# Define common lists of CNRFC sites and subwatersheds
cnrfc_site = ['BDBC1', 'EPRC1', 'ORDC1', 'HLEC1', 'FOLC1', 'EDCC1', 'TCRC1', 'MLMC1', 'DCVC1', 'BKCC1', 'BHNC1',
         'FRAC1', 'HIDC1', 'EXQC1', 'NDPC1', 'NMSC1', 'NHGC1', 'CMPC1', 'MHBC1', 'MPAC1', 'OWCC1', 'MEEC1']

subwatershed = ['Sacramento Bend', 'Stony', 'Cache', 'Upper Feather', 'Yuba', 'Bear', 'Upper American', 'Putah', 
                 'Upper Sacramento Valley', 'Sacramento Valley Floor', 'Chowchilla', 'Upper San Joaquin', 'Fresno',
                 'Merced', 'Tuolumne', 'Stanislaus', 'Calaveras', 'Mokelumne', 'Cosumnes', 'San Joaquin Valley Floor']

# 13 of the subwatersheds correspond with single CNRFC sites
match = [['Sacramento Bend', 'BDBC1'], ['Upper Feather', 'ORDC1'], ['Yuba', 'HLEC1'], ['Upper American', 'FOLC1'],
         ['Upper San Joaquin', 'FRAC1'], ['Chowchilla', 'BHNC1'], ['Fresno', 'HIDC1'], ['Merced', 'EXQC1'],
         ['Tuolumne', 'NDPC1'], ['Stanislaus', 'NMSC1'], ['Calaveras', 'NHGC1'], ['Mokelumne', 'CMPC1'], ['Cosumnes', 'MHBC1']]

## A) Past Supply

#### Specify Start and End Dates for analysis

In [ ]:
# Function to verify that the specified date is in the expected format
def verify_user_date(prompt, min_date, max_date):
    while True:
        try:
            date_input = input(prompt)
            # Parse the date using dateutil
            parsed_date = parse(date_input, dayfirst=False, yearfirst=False)
            
            # Remove time details
            parsed_date = parsed_date.replace(hour=0, minute=0, second=0, microsecond=0)

            if parsed_date < min_date or parsed_date > max_date:
                raise ValueError(f"Date must be between {min_date.strftime('%m/%d/%Y')} and {max_date.strftime('%m/%d/%Y')}.")
            
            # Display the parsed date for user confirmation
            friendly_date = parsed_date.strftime('%B %d, %Y')
            confirmation = input(f"You've entered the date: {friendly_date}. Is this correct? (Y/N): ").strip().lower()
            if confirmation in ['y', 'yes']:
                return parsed_date
            else:
                print("Please re-enter the date.")

        except ValueError as e:
            print(f"Invalid date entered: {e}. Please try again.")


# Define the earliest start date and the latest end date
earliest_start_date = datetime.strptime("10/01/2012", "%m/%d/%Y").replace(hour=0, minute=0, second=0, microsecond=0)
latest_end_date = datetime.now() + relativedelta(years=1)

# Specify user input for start and end dates
def get_dates_within_range():
    start_date = verify_user_date("Enter your start date (MM/DD/YYYY): ", earliest_start_date, latest_end_date)
    end_date = verify_user_date("Enter your end date (MM/DD/YYYY): ", earliest_start_date, latest_end_date)
    
    # Ensure start_date is before end_date
    while start_date >= end_date:
        print("The start date must be before the end date. Please enter both dates again.")
        start_date = verify_user_date("Enter your start date in the specified format(MM/DD/YYYY): ", earliest_start_date, latest_end_date)
        end_date = verify_user_date("Enter your end date in the specified format (MM/DD/YYYY): ", earliest_start_date, latest_end_date)

    return start_date, end_date

start_date, end_date = get_dates_within_range()

# Check if End date is in the past or future
current_date = datetime.now()
end_date_past = end_date if end_date < current_date else current_date - timedelta(days=1)

#### Specify Forecast Exceedance Value

In [ ]:
# The single specified exceedance probability will be used to generate forecasts for all 20 subwatersheds

# Determine the number of traces dynamically
site_forecast = pd.read_csv('https://www.cnrfc.noaa.gov/csv/BDBC1_hefs_csv_daily.csv', header = None)
num_traces = site_forecast.shape[1] - 1

def get_valid_exceedance(num_traces):
    # Calculate minimum and maximum exceedance values
    min_excd = 1 / (num_traces + 1)
    max_excd = 1 - min_excd

    while True:  # Loop until valid input is received
        try:
            # Get user input for exceedance value
            user_excd_input = input("Enter % Exceedance value: \n")
            user_excd = float(user_excd_input.strip('%')) / 100

            # Check if the exceedance value is within the allowed range or special cases
            if min_excd <= user_excd <= max_excd or user_excd in [0.01, 0.99]:
                return user_excd  # Return valid exceedance value
            else:
                print(f"ERROR! Value is too narrow. Please enter an Exceedance value in the range of {min_excd * 100:.1f}% - {max_excd * 100:.1f}% (with the exception of 1% and 99% that will yield the min and max exceedance values, respectively.)")
        except ValueError:
            print("Invalid input. Please enter a numeric value.")

user_excd = get_valid_exceedance(num_traces)

## TO DO: add functionality for the user to specify exceedances for individual subwatersheds

## TO DO: add functionality for the user to enter custom volumetric forecasts for any subwatershed

#### Store Start and End Date, and Exceedance Variables

In [ ]:
# Temporarily store date and exceedance variables, to be called in the 2.Demand.ipynb script
%store start_date
%store end_date

%store user_excd

### i) Import CNRFC past supply data

#### Links depend on the water year

In [ ]:
# Function to determine water year from an input date
def wateryear(date):
    temp = date.strftime('%Y')

    if int(date.strftime('%m')) >= 10:
        water_year = int(temp) + 1
    else:
        water_year = int(temp)
    return water_year  

# Call function for specified Start and End Dates
start_year = wateryear(start_date)
end_year = wateryear(end_date_past)

In [ ]:
# CNRFC past supply data is published by water year; all available data from each relevant year is downloaded into a list

# Initialize a list to store data for each site for each year
data_list = []
# Initialize a variable to store dates (assuming they are the same for all sites)
dates = []

# Create local variables of Start and End Date years that will be altered 
a = start_year
b = end_year

# Nested loop to gather data from multiple water years and multiple sites
# Loop through water year(s) in date range
while a <= b:
    # Temporary list to store data for the current year
    year_data = []
    
    # Loop through 22 CNRFC sites
    for x in range (22):
        # Import CSV of past data for a single site from CNRFC
        df = pd.read_csv('https://www.cnrfc.noaa.gov/ensembleProductTabularCsv.php?id='+cnrfc_site[x]+'&prodID=9&year='+str(a))
        
        # Copy past 'observed' (estimated unimpaired) values into a list
        year_data.append(df['Raw Daily Observation (TAF)'].tolist())
    
    # Store dates separetly to append later
    dates.extend(df['Date'].tolist())
    
    # Once all sites for a water year are processed, append the water year data to the main list
    data_list.extend(zip(*year_data))  # Transpose and extend the main list
    
    # Advance one year
    a += 1

# Convert the list of lists into a DataFrame
past_supply = pd.DataFrame(data_list, columns=cnrfc_site)

# Add a date column
past_supply.insert(0, 'Date', dates)
past_supply = past_supply.reset_index(drop=True)

#display(past_supply)

In [ ]:
# Convert 'Date' column to datetime
past_supply['Date'] = pd.to_datetime(past_supply['Date'])

# Pull data from the dataframe for only the specified date range
select = (past_supply['Date'] >= start_date) & (past_supply['Date'] <= end_date_past)
past_supply_range = past_supply.loc[select]
past_supply_range = past_supply_range.reset_index(drop=True)

#display(past_supply_range)

### ii) Sum Past Supply for specified period

In [ ]:
# Remove Date column first
past_supply_range = past_supply_range.iloc[: , 1:]

# Sum past supply for the specified date range, inclusive of start and end dates
# CNRFC past values are in TAF, convert to AF for analysis
supply_past_period = past_supply_range.sum()*1000

# The above will be zero if the Start Date is in the future
supply_past_period=pd.DataFrame(supply_past_period)
supply_past_period.reset_index(inplace=True)
supply_past_period.columns =['CNRFC Site', 'SUPPLY_PAST_PERIOD']

## B. Forecasted Supply

### i) Import CNRFC Forecasted Supply data

In [ ]:
# Initialize a list to store forecast trace data
all_traces = []

# Nested loop to gather data from multiple CNRFC sites and forecast traces
# Loop through 22 CNRC sites
for site_index in range (22):
    # Import CSV of forecast data for a single site from CNRFC
    site_forecast = pd.read_csv(f'https://www.cnrfc.noaa.gov/csv/{cnrfc_site[site_index]}_hefs_csv_daily.csv', header = None)
    site_forecast.columns = site_forecast.iloc[0]
    site_forecast = site_forecast.iloc[2:]
    
    # Remove time components from forecast data (each row is 1 day)
    site_forecast['GMT'] = site_forecast['GMT'].apply(lambda x: parse(x).replace(hour=0, minute=0, second=0, microsecond=0))
    site_forecast = site_forecast[(site_forecast['GMT'] >= start_date)&(site_forecast['GMT'] <= end_date)]
    site_forecast = site_forecast.reset_index(drop=True)

    # Initialize a list to store traces for the current site
    site_traces = []
    
    # Loop through forecast traces, calculating a total forecasted supply for the specified dates
    for trace_index in range (num_traces):
        total = site_forecast.iloc[:, trace_index + 1].astype(float).sum()
        
        # CNRFC forecast traces are in TCFS, convert totals to AF
        trace = total*60*60*24/43560*1000
        
        # Store the trace in the site_traces list
        site_traces.append(trace)
    
    # Add the site traces to the all_traces list
    all_traces.append(site_traces)

# Convert the all_traces list to a dataframe
Trace = pd.DataFrame(all_traces, columns=[f'Trace{i+1}' for i in range(num_traces)])

In [ ]:
# Label dataframe to match Methodology spreadsheet
Trace['CNRFC Site'] = cnrfc_site
Trace.set_index("CNRFC Site", inplace=True)

#display(Trace)

In [ ]:
## TO DO: Add functionality to intake DWR's Bulletin 120 water supply forecasts:
## https://cdec.water.ca.gov/reportapp/javareports?name=SRWSI.pdf
## https://cdec.water.ca.gov/reportapp/javareports?name=SJWSI.pdf
## https://cdec.water.ca.gov/reportapp/javareports?name=B120DIST

### ii) Calculate Supply Forecast for specified % Exceedance

In [ ]:
# Function to calculate a specified exceedance probability (q) forecast from a series (ser)
# Python equivalent of Excel's PERCENTILE.EXC function, which is used in Methodology spreadsheet
def quantile_exc(ser, q):
    ser_sorted = ser.sort_values()
    rank = q * (len(ser) + 1) - 1
    assert rank > 0, 'quantile is too small'
    rank_l = int(rank)
    # Uses Weibull plotting position formula for calculating exceedance probability
    return float(ser_sorted.iat[rank_l] + (ser_sorted.iat[rank_l + 1] - 
                                     ser_sorted.iat[rank_l]) * (rank - rank_l))

In [ ]:
user_spec_excd = pd.DataFrame()
user_spec_excd['Exceedance']=len(Trace)
user_spec_excd['CNRFC Site'] = cnrfc_site

# If a 1% Exceedance is specified, give the maximum value among the forecast traces
if user_excd == 0.01:
    maxValues = Trace.max(axis=1)
    user_spec_excd['Exceedance'] = maxValues.values

# If a 99% Exceedance is specified, give the minimum value among the forecast traces
elif user_excd == 0.99:
    minValues = Trace.min(axis=1)
    user_spec_excd['Exceedance'] = minValues.values

# For each CNRFC site, calculate a SUPPLY_PERIOD_FORECAST_CNRFC using the exceedance function
else:
    for x in range(len(Trace)):
        ser = pd.Series(Trace.iloc[x], dtype=object)
        user_spec_excd.loc[x, 'Exceedance'] = quantile_exc(ser, 1-user_excd)

#display(user_spec_excd)

### C. Total Supply

In [ ]:
# Import Gap-Filling Factors dataset
gap_filling = pd.read_csv('../user-inputs/GapFillingFactors.csv')
gap_filling = gap_filling.rename(columns={
    f'{i:02d}_RATIO': calendar.month_abbr[i] for i in range(1, 13)
})

In [ ]:
# Import Instream Flows dataset
instream_flow = pd.read_csv('../user-inputs/InstreamFlowsAbandoned.csv')
instream_flow = instream_flow.rename(columns={
    f'{i:02d}_CFS': calendar.month_abbr[i] for i in range(1, 13)
})

In [ ]:
# Function that intakes the specified Start and End Dates to calculate a multiplier
# The multiplier is used to prorate multiple months of data down to a single value for the specified dates 
def calculate_prorated_constant(start_date, end_date, constants):
    # Get the number of days in a month
    def days_in_month(year, month):
        return calendar.monthrange(year, month)[1]

    # Initialize the prorated_value
    prorated_value = 0.0

    current_date = start_date
    # Add prorated value for the start month
    prorated_value += (days_in_month(start_date.year, start_date.month) - start_date.day + 1) / days_in_month(start_date.year, start_date.month) * constants[start_date.month - 1]

    # Increment month by month until end date is reached
    while True:
        next_month_first_day = (current_date.replace(day=28) + timedelta(days=4)).replace(day=1) # To safely get the first day of next month
        if next_month_first_day >= end_date:
            break
        
        # Add prorated value for full month or fraction for partial month
        if next_month_first_day.month == end_date.month:
            prorated_value += end_date.day / days_in_month(end_date.year, end_date.month) * constants[end_date.month - 1]
        else:
            prorated_value += constants[next_month_first_day.month - 1]
        current_date = next_month_first_day

    # Calculate the number of days in the range
    number_of_days = (end_date - start_date).days + 1  # plus one to include the end date

    return prorated_value, number_of_days

In [ ]:
# Call the proration function to transform monthly Gap-Filling Factors into single Period factors based on specified Start and End Dates
gap_list = []

for gapfilling in gap_filling['GAP_FILLING_FACTOR']:
    constant, number_of_days = calculate_prorated_constant(start_date, end_date, gap_filling.loc[gap_filling['GAP_FILLING_FACTOR'] == gapfilling].drop('GAP_FILLING_FACTOR', axis=1).squeeze())
    gap_list.append({'Period_GAP_Constant': constant, 'GAP_FILLING_FACTOR': gapfilling})

period_gap = pd.DataFrame(gap_list)

#display(period_gap)

In [ ]:
# Call the proration function to transform monthly Instream Flows into single Period flow volumes based on specified Start and End Dates
inf_list = []

for subwatersheds in instream_flow['SUBWATERSHED']:
    constant, number_of_days = calculate_prorated_constant(start_date, end_date, instream_flow.loc[instream_flow['SUBWATERSHED'] == subwatersheds].drop('SUBWATERSHED', axis=1).squeeze())
    
    # Convert the instream flow rates (CFS) to volummes (AF)
    inf_list.append({'INSTREAM_FLOW_PERIOD': constant*number_of_days*60*60*24/43560, 'SUBWATERSHED': subwatersheds})

period_inf = pd.DataFrame(inf_list)

#display(period_inf)

In [ ]:
# Export intermediate results for use in 3.Analysis script
period_inf.to_csv('./intermediate-outputs/Instream_Flow_Period.csv', index=False)

#### Sum Past and Forecasted Supply to calculate Supply_Period_Total

In [ ]:
# Set the Supply Forecast to the specified exceedance
supply_period_forecast_cnrfc = user_spec_excd.rename(columns = {'Exceedance':'SUPPLY_PERIOD_FORECAST_CNRFC'})
supply_period_forecast_cnrfc = supply_period_forecast_cnrfc[['CNRFC Site', 'SUPPLY_PERIOD_FORECAST_CNRFC']]
# The above will be zero if the End Date is in the past

In [ ]:
# Add the SUPPLY_PAST_PERIOD and SUPPLY_PERIOD_FORECAST_SELECTED values
supply_period_total_cnrfc = pd.merge(supply_past_period, supply_period_forecast_cnrfc, on=['CNRFC Site'])
supply_period_total_cnrfc['SUPPLY_PERIOD_TOTAL'] = supply_period_total_cnrfc['SUPPLY_PAST_PERIOD'] + supply_period_total_cnrfc['SUPPLY_PERIOD_FORECAST_CNRFC']

# Zero out any negative values in 'SUPPLY_PAST_TOTAL' (negative computed unimpaired flows are summed as-is)
supply_period_total_cnrfc.loc[supply_period_total_cnrfc['SUPPLY_PERIOD_TOTAL'] < 0, 'SUPPLY_PERIOD_TOTAL'] = 0

#### Assign CNRFC supply to 13 subwatersheds associated with single sites

In [ ]:
supply_period_total = pd.DataFrame(subwatershed, columns=['SUBWATERSHED'])
supply_period_total['SUPPLY_PERIOD_TOTAL'] = 0

# Convert 'SUPPLY_PERIOD_TOTAL' to float64 if it's not already
supply_period_total['SUPPLY_PERIOD_TOTAL'] = supply_period_total['SUPPLY_PERIOD_TOTAL'].astype(float)

# Call the dataframe created at the start to look up these 13 subwatersheds
for x in range (13):
    supply_period_total.loc[supply_period_total['SUBWATERSHED'] == match[x][0], 'SUPPLY_PERIOD_TOTAL'] = \
    supply_period_total_cnrfc.loc[supply_period_total_cnrfc['CNRFC Site'] == match[x][1], 'SUPPLY_PERIOD_TOTAL'].values[0]

#### Calculate supply for the remaining 7 subwatersheds (sum of multiple sites, augmentation, and/or extrapolation)

In [ ]:
# Stony is CNRFC EPRC1 augmented
supply_period_total.loc[supply_period_total['SUBWATERSHED'] == 'Stony', 'SUPPLY_PERIOD_TOTAL'] = \
supply_period_total_cnrfc.loc[supply_period_total_cnrfc['CNRFC Site'] == 'EPRC1', 'SUPPLY_PERIOD_TOTAL'].values[0] * \
period_gap.loc[period_gap['GAP_FILLING_FACTOR'] == 'Augment_EPRC1', 'Period_GAP_Constant'].values[0]

# Cache is extrapolated based on Stony
supply_period_total.loc[supply_period_total['SUBWATERSHED'] == 'Cache', 'SUPPLY_PERIOD_TOTAL'] = \
supply_period_total_cnrfc.loc[supply_period_total_cnrfc['CNRFC Site'] == 'EPRC1', 'SUPPLY_PERIOD_TOTAL'].values[0] * \
period_gap.loc[period_gap['GAP_FILLING_FACTOR'] == 'Augment_EPRC1', 'Period_GAP_Constant'].values[0] * \
period_gap.loc[period_gap['GAP_FILLING_FACTOR'] == 'Extrapolate_Cache_Stony', 'Period_GAP_Constant'].values[0]

# Bear is extrapolated based on Yuba
supply_period_total.loc[supply_period_total['SUBWATERSHED'] == 'Bear', 'SUPPLY_PERIOD_TOTAL'] = \
supply_period_total_cnrfc.loc[supply_period_total_cnrfc['CNRFC Site'] == 'HLEC1', 'SUPPLY_PERIOD_TOTAL'].values[0] * \
period_gap.loc[period_gap['GAP_FILLING_FACTOR'] == 'Extapolate_Bear_HLEC1', 'Period_GAP_Constant'].values[0]

# Putah is extrapolated based on Stony
supply_period_total.loc[supply_period_total['SUBWATERSHED'] == 'Putah', 'SUPPLY_PERIOD_TOTAL'] = \
supply_period_total_cnrfc.loc[supply_period_total_cnrfc['CNRFC Site'] == 'EPRC1', 'SUPPLY_PERIOD_TOTAL'].values[0] * \
period_gap.loc[period_gap['GAP_FILLING_FACTOR'] == 'Augment_EPRC1', 'Period_GAP_Constant'].values[0] * \
period_gap.loc[period_gap['GAP_FILLING_FACTOR'] == 'Extrapolate_Putah_Stony', 'Period_GAP_Constant'].values[0]

In [ ]:
# Upper Sacramento Valley is CNRFC (EDCC1+TCRC1 augmented) + (MLMC1+DCVC1+BKCC1 augmented)
supply_period_total.loc[supply_period_total['SUBWATERSHED'] == 'Upper Sacramento Valley', 'SUPPLY_PERIOD_TOTAL'] = \
((supply_period_total_cnrfc.loc[supply_period_total_cnrfc['CNRFC Site'] == 'EDCC1', 'SUPPLY_PERIOD_TOTAL'].values[0] + \
supply_period_total_cnrfc.loc[supply_period_total_cnrfc['CNRFC Site'] == 'TCRC1', 'SUPPLY_PERIOD_TOTAL'].values[0]) * \
period_gap.loc[period_gap['GAP_FILLING_FACTOR'] == 'Augment_EDCC1+TCRC1', 'Period_GAP_Constant'].values[0]) + \
((supply_period_total_cnrfc.loc[supply_period_total_cnrfc['CNRFC Site'] == 'MLMC1', 'SUPPLY_PERIOD_TOTAL'].values[0] + \
  supply_period_total_cnrfc.loc[supply_period_total_cnrfc['CNRFC Site'] == 'DCVC1', 'SUPPLY_PERIOD_TOTAL'].values[0] + \
  supply_period_total_cnrfc.loc[supply_period_total_cnrfc['CNRFC Site'] == 'BKCC1', 'SUPPLY_PERIOD_TOTAL'].values[0]) * \
 period_gap.loc[period_gap['GAP_FILLING_FACTOR'] == 'Augment_MLMC1+DCVC1+BKCC1', 'Period_GAP_Constant'].values[0])

# Sacramento Valley Floor is extrapolated based on (Sacramento Bend + Upper Feather + Upper American)
supply_period_total.loc[supply_period_total['SUBWATERSHED'] == 'Sacramento Valley Floor', 'SUPPLY_PERIOD_TOTAL'] = \
((supply_period_total_cnrfc.loc[supply_period_total_cnrfc['CNRFC Site'] == 'BDBC1', 'SUPPLY_PERIOD_TOTAL'].values[0] + \
  supply_period_total_cnrfc.loc[supply_period_total_cnrfc['CNRFC Site'] == 'ORDC1', 'SUPPLY_PERIOD_TOTAL'].values[0] + \
  supply_period_total_cnrfc.loc[supply_period_total_cnrfc['CNRFC Site'] == 'FOLC1', 'SUPPLY_PERIOD_TOTAL'].values[0]) * \
  period_gap.loc[period_gap['GAP_FILLING_FACTOR'] == 'Extrapolate_SacramentoValleyFloor_BDBC1+ORDC1+FOLC1', 'Period_GAP_Constant'].values[0])

# San Joaquin Valley FLoor is (CNRFC MPAC1+OWCC1+MEEC1) + (extrapolated based on Upper San Joaquin + Merced + Tuolumne
# + Stanislaus) + (extrapolated based on Mokelumne + Cosumnes)
supply_period_total.loc[supply_period_total['SUBWATERSHED'] == 'San Joaquin Valley Floor', 'SUPPLY_PERIOD_TOTAL'] = \
supply_period_total_cnrfc.loc[supply_period_total_cnrfc['CNRFC Site'] == 'MPAC1', 'SUPPLY_PERIOD_TOTAL'].values[0] + \
supply_period_total_cnrfc.loc[supply_period_total_cnrfc['CNRFC Site'] == 'OWCC1', 'SUPPLY_PERIOD_TOTAL'].values[0] + \
supply_period_total_cnrfc.loc[supply_period_total_cnrfc['CNRFC Site'] == 'MEEC1', 'SUPPLY_PERIOD_TOTAL'].values[0] + \
((supply_period_total_cnrfc.loc[supply_period_total_cnrfc['CNRFC Site'] == 'FRAC1', 'SUPPLY_PERIOD_TOTAL'].values[0] + \
  supply_period_total_cnrfc.loc[supply_period_total_cnrfc['CNRFC Site'] == 'EXQC1', 'SUPPLY_PERIOD_TOTAL'].values[0] + \
  supply_period_total_cnrfc.loc[supply_period_total_cnrfc['CNRFC Site'] == 'NDPC1', 'SUPPLY_PERIOD_TOTAL'].values[0] + \
  supply_period_total_cnrfc.loc[supply_period_total_cnrfc['CNRFC Site'] == 'NMSC1', 'SUPPLY_PERIOD_TOTAL'].values[0]) * \
  period_gap.loc[period_gap['GAP_FILLING_FACTOR'] == 'Extrapolate_SanJoaquinValleyFloor_FRAC1+EXQC1+NDPC1+NMSC1', 'Period_GAP_Constant'].values[0]) + \
((supply_period_total_cnrfc.loc[supply_period_total_cnrfc['CNRFC Site'] == 'CMPC1', 'SUPPLY_PERIOD_TOTAL'].values[0] + \
  supply_period_total_cnrfc.loc[supply_period_total_cnrfc['CNRFC Site'] == 'MHBC1', 'SUPPLY_PERIOD_TOTAL'].values[0]) * \
 period_gap.loc[period_gap['GAP_FILLING_FACTOR'] == 'Extrapolate_SanJoaquinValleyFloor_CMPC1+MHBC1', 'Period_GAP_Constant'].values[0])

In [ ]:
# Export intermediate results for use in 3.Analysis script
supply_period_total.to_csv('./intermediate-outputs/Supply_Period_Total.csv', index=False)